# **-------------------------------Attendance Report Generation-------------------------------**

### **📌 Step 1: Import Libraries**

In [7]:
import pandas as pd
import urllib.parse
from sqlalchemy import create_engine, text
import logging

# Configure logging
logging.basicConfig(filename=r"I:\My Drive\Kotak Salesian School\attendance_report.log", level=logging.ERROR, 
                    format="%(asctime)s - %(levelname)s - %(message)s")


### **📌 Step 2: Define MySQL Credentials & Table Name**

In [53]:
# MySQL Credentials
MYSQL_CREDENTIALS = {
    "username": "root",
    "password": "Hari@123",
    "host": "localhost",
    "port": "3306",
    "database": "schooldb",
}
TABLE_NAME = "attendance_report1"


### **📌 Step 3: Load and Clean Data**

In [108]:
import pandas as pd

# File Paths
file1 = r"I:\My Drive\Kotak Salesian School\2024-25\Attendance Report\AttendanceReportUptoSeptember.csv"
file2 = r"I:\My Drive\Kotak Salesian School\2024-25\Attendance Report\AttendanceOctoberToDecember.csv"
file3 = r"I:\My Drive\Kotak Salesian School\2024-25\Attendance Report\AttendanceUptoFebruary.csv"
output_file = r"I:\My Drive\Kotak Salesian School\2024-25\Attendance Report\AttendanceReport-copy.csv"

# Load Data
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

# Standardize column names
for df in [df1, df2, df3]:
    df.columns = df.columns.str.strip().str.replace('"', '', regex=False)

# Merge DataFrames on 'Students Number' using outer join
df = df1.merge(df2, on='Students Number', how='outer').merge(df3, on='Students Number', how='outer')

# Identify and handle duplicate columns
common_fields = ['Name', 'Class']
for field in common_fields:
    df[field] = df.pop(f"{field}_x").combine_first(df.pop(f"{field}_y"))

# Drop remaining duplicate columns
drop_columns = [col for col in df.columns if '_x' in col or '_y' in col]
df = df.drop(columns=drop_columns, errors='ignore')

# Rename 'Students Number' to 'AdmissionNo'
df = df.rename(columns={"Students Number": "AdmissionNo"})

# Reorder Columns
column_order = ['AdmissionNo', 'Name', 'Class'] + [col for col in df.columns if col not in ['AdmissionNo', 'Name', 'Class']]
df = df[column_order]

# Drop Unnecessary Columns
columns_to_drop = ["Present Days", "Absent Days", "Toral Working Days"]  # Ensure correct column names
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

df


,AdmissionNo,Name,Class,17.07.2024,18.07.2024,19.07.2024,20.07.2024,21.07.2024,22.07.2024,23.07.2024,...,29.01.2025,30.01.2025,31.01.2025,01.02.2025,02.02.2025,03.02.2025,04.02.2025,05.02.2025,06.02.2025,07.02.2025
0,13553,TRIVENI CHANDANALA (W.H),ICSE (X - C),H,P,H,H,H,P,P,...,P,P,P,P,H,P,P,P,P,P
1,13566,MOHAMMAD TALHA ANSARI,ICSE (X - A),H,A,H,H,H,P,P,...,P,P,P,P,H,A,P,P,P,P
2,13578,RISHITHA AKKARABOINA(W.H),ICSE (X - C),H,P,H,H,H,P,P,...,P,P,P,P,H,P,A,A,A,A
3,13613,PREM ROSHAN ASAPU(W.H),ICSE (X - C),H,P,H,H,H,P,P,...,A,A,A,P,H,P,A,A,P,A
4,13625,RAHUL L.,ICSE (X - B),H,A,H,H,H,P,P,...,P,P,P,P,H,A,A,P,P,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1775,D-17085,IVAAN BARLA,ICSE (Pre KG - ),H,P,H,H,H,A,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1776,D-17087,DHANYA ANSLY PALLI,ICSE (V - A),H,P,H,H,H,H,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1777,D-17129,ARNI MOKSHITH DOKI,ICSE (I - D),H,P,H,H,H,A,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1778,SKL001,Praveen_test,ICSE (VIII - C),H,P,H,H,H,P,P,...,H,H,H,H,H,H,H,H,H,H


### **📌 Step 4: Process Attendance Data**

In [109]:
# Step 1: Clean 'AdmissionNo'
df = df[~(df["AdmissionNo"].astype(str) == "786") & ~df["AdmissionNo"].astype(str).str.match(r"^[a-zA-Z]")].copy()

# Step 2: Extract Class and Section
df["Class"] = df["Class"].astype(str).str.replace(r"ICSE \((.*?)\)", r"\1", regex=True)

df

,AdmissionNo,Name,Class,17.07.2024,18.07.2024,19.07.2024,20.07.2024,21.07.2024,22.07.2024,23.07.2024,...,29.01.2025,30.01.2025,31.01.2025,01.02.2025,02.02.2025,03.02.2025,04.02.2025,05.02.2025,06.02.2025,07.02.2025
0,13553,TRIVENI CHANDANALA (W.H),X - C,H,P,H,H,H,P,P,...,P,P,P,P,H,P,P,P,P,P
1,13566,MOHAMMAD TALHA ANSARI,X - A,H,A,H,H,H,P,P,...,P,P,P,P,H,A,P,P,P,P
2,13578,RISHITHA AKKARABOINA(W.H),X - C,H,P,H,H,H,P,P,...,P,P,P,P,H,P,A,A,A,A
3,13613,PREM ROSHAN ASAPU(W.H),X - C,H,P,H,H,H,P,P,...,A,A,A,P,H,P,A,A,P,A
4,13625,RAHUL L.,X - B,H,A,H,H,H,P,P,...,P,P,P,P,H,A,A,P,P,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757,17160,HARI CHANDANA MARDANA,Pre KG -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,P,P,P,P,H,P,P,P,P,P
1758,17161,NATHAN SOLOMON MARIA PERIMAL ANTHONY,UKG - C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,A,A,A,H,A,A,A,A,A
1759,17162,KEETHI KUSHVIKA BONGU,Pre KG -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,A,P,P,H,P,P,P,P,P
1760,17163,MUHAMMAD ZAYN KHAN,Pre KG -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,P,P,A,A,H,P,A,P,P,P


In [110]:
# Step 3: Unpivot DataFrame
df_unpivot = pd.melt(df, id_vars=["AdmissionNo", "Name", "Class"], 
                     var_name="Date", value_name="AttendanceStatus")

# Step 4: Convert 'Date' to datetime
df_unpivot["Date"] = pd.to_datetime(df_unpivot["Date"], format='%d.%m.%Y', errors='coerce')

# Step 5: Log invalid 'Date' values
if df_unpivot["Date"].isna().sum() > 0:
    print("⚠️ Warning: Some Date values were invalid and converted to NaT.")
    missing_dates = df_unpivot[df_unpivot["Date"].isna()]
    print("⚠️ Warning: The following rows have invalid dates:")
    print(missing_dates.to_string())

df_unpivot

,AdmissionNo,Name,Class,Date,AttendanceStatus
0,13553,TRIVENI CHANDANALA (W.H),X - C,2024-07-17,H
1,13566,MOHAMMAD TALHA ANSARI,X - A,2024-07-17,H
2,13578,RISHITHA AKKARABOINA(W.H),X - C,2024-07-17,H
3,13613,PREM ROSHAN ASAPU(W.H),X - C,2024-07-17,H
4,13625,RAHUL L.,X - B,2024-07-17,H
...,...,...,...,...,...
362967,17160,HARI CHANDANA MARDANA,Pre KG -,2025-02-07,P
362968,17161,NATHAN SOLOMON MARIA PERIMAL ANTHONY,UKG - C,2025-02-07,A
362969,17162,KEETHI KUSHVIKA BONGU,Pre KG -,2025-02-07,P
362970,17163,MUHAMMAD ZAYN KHAN,Pre KG -,2025-02-07,P


In [111]:
df_unpivot = df_unpivot[~df_unpivot["AttendanceStatus"].eq("H") & (df_unpivot["Date"] != "2024-07-17")].reset_index(drop=True)

df_unpivot

,AdmissionNo,Name,Class,Date,AttendanceStatus
0,13553,TRIVENI CHANDANALA (W.H),X - C,2024-07-18,P
1,13566,MOHAMMAD TALHA ANSARI,X - A,2024-07-18,A
2,13578,RISHITHA AKKARABOINA(W.H),X - C,2024-07-18,P
3,13613,PREM ROSHAN ASAPU(W.H),X - C,2024-07-18,P
4,13625,RAHUL L.,X - B,2024-07-18,A
...,...,...,...,...,...
234129,17160,HARI CHANDANA MARDANA,Pre KG -,2025-02-07,P
234130,17161,NATHAN SOLOMON MARIA PERIMAL ANTHONY,UKG - C,2025-02-07,A
234131,17162,KEETHI KUSHVIKA BONGU,Pre KG -,2025-02-07,P
234132,17163,MUHAMMAD ZAYN KHAN,Pre KG -,2025-02-07,P


In [112]:
# Step 6: Sort by 'AdmissionNo' and reset index
df_unpivot = df_unpivot.sort_values("Date", ascending=False).reset_index(drop=True)

# Step 8: Identify students with P, A, or H
students_with_attendance = df_unpivot[df_unpivot['AttendanceStatus'].isin(["P", "A", "H"])]['AdmissionNo'].unique()


In [113]:
# Step 9: Assign 'TC' if student has previous attendance records
df_unpivot['AttendanceStatus'] = df_unpivot.apply(
    lambda row: "TC" if pd.isna(row['AttendanceStatus']) and row['AdmissionNo'] in students_with_attendance else row['AttendanceStatus'],
    axis=1
)

df_unpivot

,AdmissionNo,Name,Class,Date,AttendanceStatus
0,17164,SHAIK REHAN SIDDIQUE,LKG - A,2025-02-07,P
1,15552,SAI POOJITHA KOVIRI,V - B,2025-02-07,P
2,15538,VENKATA VIGNESWARI SAI VINYA BOTTA,V - C,2025-02-07,A
3,15539,MUSKHAN KARRI,V - B,2025-02-07,P
4,15540,HARSHAVARDHAN KOYYAPU,V - C,2025-02-07,P
...,...,...,...,...,...
234129,16527,NAVADEEP MOOGI,VII - C,2024-07-18,P
234130,16528,KRISHNA JESSIKA MAKKA,V - B,2024-07-18,P
234131,16529,GEETHIKA SAANVI KOLLI,I - B,2024-07-18,P
234132,16530,UDAY SATWIK PAKKI,I - B,2024-07-18,P


In [114]:
df_unpivot[df_unpivot["AttendanceStatus"].isna()]

,AdmissionNo,Name,Class,Date,AttendanceStatus


In [115]:
# Step 11: Prioritize Attendance Status
priority_map = {'P': 2, 'A': 1, 'H': 3, 'Not Joined': 4, 'TC': 5}

df_unpivot['Priority'] = df_unpivot["AttendanceStatus"].map(priority_map)

df_unpivot = df_unpivot.sort_values(by=['AdmissionNo', 'Date', 'Priority']) \
                        .drop_duplicates(subset=['AdmissionNo', 'Date'], keep='first') \
                        .drop(columns=['Priority'])

df_unpivot

,AdmissionNo,Name,Class,Date,AttendanceStatus
234133,13553,TRIVENI CHANDANALA (W.H),X - C,2024-07-18,P
231068,13553,TRIVENI CHANDANALA (W.H),X - C,2024-07-22,P
229179,13553,TRIVENI CHANDANALA (W.H),X - C,2024-07-23,P
227709,13553,TRIVENI CHANDANALA (W.H),X - C,2024-07-24,P
225816,13553,TRIVENI CHANDANALA (W.H),X - C,2024-07-25,P
...,...,...,...,...,...
8217,17164,SHAIK REHAN SIDDIQUE,LKG - A,2025-02-03,P
6457,17164,SHAIK REHAN SIDDIQUE,LKG - A,2025-02-04,P
4839,17164,SHAIK REHAN SIDDIQUE,LKG - A,2025-02-05,P
2940,17164,SHAIK REHAN SIDDIQUE,LKG - A,2025-02-06,P


In [116]:
# Step 12: Final sorting and formatting
df_unpivot = df_unpivot[['Date', 'AdmissionNo', 'Name', 'Class', 'AttendanceStatus']]
df_unpivot.sort_values(by=['Date'], ascending=False, inplace=True)

df_unpivot

,Date,AdmissionNo,Name,Class,AttendanceStatus
0,2025-02-07,17164,SHAIK REHAN SIDDIQUE,LKG - A,P
294,2025-02-07,16122,VISWA GUBBALA,V - C,P
447,2025-02-07,14311,PRIYA TEDDU,IX - B,P
293,2025-02-07,16121,JAGRUK CHOWDHURY,V - C,A
1534,2025-02-07,16253,NITHYA ASHRAYA METIKOTI,IV - A,P
...,...,...,...,...,...
233331,2024-07-18,16910,AYAAN JOEL BATTULA,UKG - A,P
233113,2024-07-18,15189,JASWIKA PYLA,VI - A,P
232496,2024-07-18,15593,NISHANTH SINGAMPALLI,V - D,P
233691,2024-07-18,16998,MARTIN VELANGINI SUPRIYA,LKG - B,P


In [117]:
df_unpivot['Class'] = df_unpivot['Class'].str.replace("Pre KG - ", "Pre KG")

# Step 13: Replace Attendance Status with meaningful labels
df_unpivot["AttendanceStatus"] = df_unpivot["AttendanceStatus"].replace({
    'P': "Present", 'A': "Absent", 'H': "Holiday", 'Not Joined': "Not Joined", 'TC': "TC"
})

# Extract unique holidays and merge back with non-holiday data
df_unpivot = pd.concat([
    df_unpivot[df_unpivot["AttendanceStatus"].isin(["Present", "Absent", "TC", "Not Joined"])],
    df_unpivot[df_unpivot["AttendanceStatus"] == "Holiday"].drop_duplicates(subset=["Date"])
])

df_unpivot = df_unpivot.reset_index(drop=True)

df_unpivot

,Date,AdmissionNo,Name,Class,AttendanceStatus
0,2025-02-07,17164,SHAIK REHAN SIDDIQUE,LKG - A,Present
1,2025-02-07,16122,VISWA GUBBALA,V - C,Present
2,2025-02-07,14311,PRIYA TEDDU,IX - B,Present
3,2025-02-07,16121,JAGRUK CHOWDHURY,V - C,Absent
4,2025-02-07,16253,NITHYA ASHRAYA METIKOTI,IV - A,Present
...,...,...,...,...,...
234129,2024-07-18,16910,AYAAN JOEL BATTULA,UKG - A,Present
234130,2024-07-18,15189,JASWIKA PYLA,VI - A,Present
234131,2024-07-18,15593,NISHANTH SINGAMPALLI,V - D,Present
234132,2024-07-18,16998,MARTIN VELANGINI SUPRIYA,LKG - B,Present


In [119]:
# Create a dictionary for Class and Section to their respective unique numbers
class_section_mapping = {
"Pre KG": 1, "LKG - A": 2, "LKG - B": 3, "UKG - A": 4, "UKG - B": 5, "UKG - C": 6,
"I - A": 7, "I - B": 8, "I - C": 9, "I - D": 10, "II - A": 11, "II - B": 12, "II - C": 13, "II - D": 14,
"III - A": 15, "III - B": 16, "III - C": 17, "III - D": 18, "IV - A": 19, "IV - B": 20, "IV - C": 21, "IV - D": 22,
"V - A": 23, "V - B": 24, "V - C": 25, "V - D": 26, "VI - A": 27, "VI - B": 28, "VI - C": 29, "VI - D": 30,
"VII - A": 31, "VII - B": 32, "VII - C": 33, "VII - D": 34, "VIII - A": 35, "VIII - B": 36, "VIII - C": 37,
"IX - A": 38, "IX - B": 39, "IX - C": 40, "X - A": 41, "X - B": 42, "X - C": 43}

# Create the 'ClassNo' column using the mapping
df_unpivot['ClassNo'] = df_unpivot['Class'].map(class_section_mapping)


# Define the grade mapping
grade_mapping = [
("Pre KG", 1), ("LKG", 2), ("UKG", 3),
("I", 4), ("II", 5), ("III", 6), ("IV", 7), ("V", 8),
("VI", 9), ("VII", 10), ("VIII", 11), ("IX", 12), ("X", 13)
]

# Create conditions for grade extraction (checking if the class contains a grade name)
conditions = [df_unpivot['Class'].str.contains(fr"\b{k}\b", na=False, regex=True) for k, _ in grade_mapping]

# Extract grade IDs
choices = [v for _, v in grade_mapping]

# Apply np.select to create the 'gradeId' column
df_unpivot['gradeId'] = np.select(conditions, choices, default=0)


AttendanceStatus_mapping = [("Absent", 1), ("Present", 2), ("TC", 3), ("Holiday", 4)]

conditions = [df_unpivot['AttendanceStatus'].str.contains(k, na=False) for k, _ in AttendanceStatus_mapping]
choices = [v for _, v in AttendanceStatus_mapping]

df_unpivot['AttendanceStatusId'] = np.select(conditions, choices, default=0)

# Define branch mapping as a dictionary
branch_mapping = [
('Pre KG',1), ('LKG', 1), ('UKG',1),
('I',2,), ('II',2), ('III',2), ('IV',2), ('V',2),
('VI',3), ('VII',3,),('VIII',3), ('IX',3), ('X',3)
]

# Create conditions for grade extraction (checking if the class contains a grade name)
conditions = [df_unpivot['Class'].str.contains(fr"\b{k}\b", na=False, regex=True) for k, _ in grade_mapping]
choices = [v for _, v in branch_mapping]

df_unpivot['branchId'] = np.select(conditions, choices, default=0)

import numpy as np

# Define branch mapping as a dictionary
branch_mapping = {
    1: 'Kindergarten',2: 'Primary',3: 'Higher'}

# Convert 'branchId' column to string if it's not already
df_unpivot['branchId'] = df_unpivot['branchId'].astype(str)

# Create conditions for branch extraction
conditions = [df_unpivot['branchId'] == str(k) for k in branch_mapping.keys()]
choices = list(branch_mapping.values())

# Apply np.select to create a new column 'branchName'
df_unpivot['branchName'] = np.select(conditions, choices, default="Unknown")

# ✅ Now df_unpivot['branchName'] will contain branch names based on branchId


# Apply mapping using np.select

df_unpivot = df_unpivot.sort_values(by=['Date'], ascending=False)

df_unpivot = df_unpivot[['Date', 'AdmissionNo', 'Name', 'Class', 'ClassNo', 'gradeId', 'branchId',"branchName", 'AttendanceStatus', 'AttendanceStatusId']]

df_unpivot


,Date,AdmissionNo,Name,Class,ClassNo,gradeId,branchId,branchName,AttendanceStatus,AttendanceStatusId
0,2025-02-07,17164,SHAIK REHAN SIDDIQUE,LKG - A,2,2,1,Kindergarten,Present,2
15,2025-02-07,16732,BAVIKA SETTY,III - D,18,6,2,Primary,Present,2
4,2025-02-07,16253,NITHYA ASHRAYA METIKOTI,IV - A,19,7,2,Primary,Present,2
5,2025-02-07,14990,IRFAN AHMED M A,VII - D,34,10,3,Higher,Present,2
6,2025-02-07,16120,ROSARY ANTHONY TANDU,IV - B,20,7,2,Primary,Present,2
...,...,...,...,...,...,...,...,...,...,...
234129,2024-07-18,16910,AYAAN JOEL BATTULA,UKG - A,4,3,1,Kindergarten,Present,2
234130,2024-07-18,15189,JASWIKA PYLA,VI - A,27,9,3,Higher,Present,2
234131,2024-07-18,15593,NISHANTH SINGAMPALLI,V - D,26,8,2,Primary,Present,2
234132,2024-07-18,16998,MARTIN VELANGINI SUPRIYA,LKG - B,3,2,1,Kindergarten,Present,2


In [107]:
df_unpivot

,Date,AdmissionNo,Name,Class,ClassNo,gradeId,branchId,AttendanceStatus,AttendanceStatusId
0,2025-02-07,17164,SHAIK REHAN SIDDIQUE,LKG - A,2,2,1,Present,2
15,2025-02-07,16732,BAVIKA SETTY,III - D,18,6,2,Present,2
4,2025-02-07,16253,NITHYA ASHRAYA METIKOTI,IV - A,19,7,2,Present,2
5,2025-02-07,14990,IRFAN AHMED M A,VII - D,34,10,3,Present,2
6,2025-02-07,16120,ROSARY ANTHONY TANDU,IV - B,20,7,2,Present,2
...,...,...,...,...,...,...,...,...,...
234129,2024-07-18,16910,AYAAN JOEL BATTULA,UKG - A,4,3,1,Present,2
234130,2024-07-18,15189,JASWIKA PYLA,VI - A,27,9,3,Present,2
234131,2024-07-18,15593,NISHANTH SINGAMPALLI,V - D,26,8,2,Present,2
234132,2024-07-18,16998,MARTIN VELANGINI SUPRIYA,LKG - B,3,2,1,Present,2


### **📌 Step 5: Insert Data into MySQL**

In [11]:
def update_database(df):
    """Insert attendance data into MySQL database."""
    password = urllib.parse.quote(MYSQL_CREDENTIALS["password"])
    engine = create_engine(f"mysql+pymysql://{MYSQL_CREDENTIALS['username']}:{password}"
                           f"@{MYSQL_CREDENTIALS['host']}:{MYSQL_CREDENTIALS['port']}/{MYSQL_CREDENTIALS['database']}")
    
    try:
        print(f"Connecting to database {MYSQL_CREDENTIALS['database']} at {MYSQL_CREDENTIALS['host']}...\n")
        
        with engine.begin() as conn:
            print(f"Truncating existing table: {TABLE_NAME}")
            conn.execute(text(f"TRUNCATE TABLE {TABLE_NAME};"))
            print(f"Inserting data into {TABLE_NAME} table...\n")
            
            # Try inserting the data in chunks (optional to prevent overload)
            df.to_sql(name=TABLE_NAME, con=engine, if_exists='append', index=False, chunksize=1000)  # Chunks of 1000 rows
            print(f"✅ Data successfully inserted into '{TABLE_NAME}' table.\n")
    
    except Exception as e:
        print(f"❌ An error occurred. Check the logs for details. Error: {e}")
        logging.error(f"❌ Database update failed: {e}")
        logging.error(f"❌ Failed Data Sample (first 5 rows): \n{df.head()}")
        logging.error(f"Total Rows in DataFrame: {df.shape[0]}")
        
        # Capture more details to help debug
        logging.error("MySQL Connection Information:")
        logging.error(f"Host: {MYSQL_CREDENTIALS['host']}")
        logging.error(f"Database: {MYSQL_CREDENTIALS['database']}")
        logging.error(f"Port: {MYSQL_CREDENTIALS['port']}")
        logging.error("Error Traceback:")
        import traceback
        logging.error(traceback.format_exc())


### **📌 Step 6: Run the Full Pipeline**

In [12]:
def main():
    file1 = r"I:\My Drive\Kotak Salesian School\2024-25\Attendance Report\AttendanceReportUptoSeptember.csv"
    file2 = r"I:\My Drive\Kotak Salesian School\2024-25\Attendance Report\AttendanceOctoberToDecember.csv"
    file3 = r"I:\My Drive\Kotak Salesian School\2024-25\Attendance Report\AttendanceUptoFebruary.csv"
    output_file = r"I:\My Drive\Kotak Salesian School\2024-25\Attendance Report\AttendanceReport-copy.csv"
    
    try:
        print("Loading and cleaning data...\n")
        df = load_and_clean_data(file1, file2, file3)
        print(f"✅ Data loaded with {df.shape[0]} rows.\n")
        
        print("Processing attendance data...\n")
        df_unpivot = process_attendance_data(df)
        df_unpivot.to_csv(output_file, index=False)
        print(f"✅ Processed data with {df_unpivot.shape[0]} rows.\n")
        print("✅ Columns are:\n", df_unpivot.columns)
        # print(df_unpivot[df_unpivot["Date"] == "2025-02-03"].to_string())
                
        print("Updating database...\n")
        update_database(df_unpivot)
        print("✅ Data updated successfully!\n")

        print("✅ Attendance report processing completed successfully!\n")
        print(f"✅ No of Rows: {df_unpivot.shape[0]}\n")
    except Exception as e:
        print(f"❌ An unexpected error occurred. Error: {e}\n")
        logging.error(f"❌ Unexpected error: {e}\n")


# Run the script
main()


Loading and cleaning data...

✅ Data loaded with 1780 rows.

Processing attendance data...

✅ Processed data with 236398 rows.

✅ Columns are:
 Index(['Date', 'AdmissionNo', 'Name', 'Class', 'AttendanceStatus', 'ClassNo',
       'gradeId', 'AttendanceStatusId', 'branchId'],
      dtype='object')
Updating database...

Connecting to database schooldb at localhost...

Truncating existing table: attendance_report1
Inserting data into attendance_report1 table...

✅ Data successfully inserted into 'attendance_report1' table.

✅ Data updated successfully!

✅ Attendance report processing completed successfully!

✅ No of Rows: 236398

